# `GenericSeries` Tutorial

## Terminology

**Dimension**

Each dimension describes a single degree of freedom. We can think of it as a 1d-coordinate axis. Using multiple dimensions will create a multi-dimensional space. A typical example would be the dimensions $x$, $y$, and $z$ that form 3d-space. Another popular dimension is time.

**Coordinates**

A coordinate is a specific value or label on the 1d-axis of a dimension. We can specify the location of a point in 3d-space by providing its coordinates. For example, we can use $x=1m$, $y=3m$, and $z=0m$. It is important to understand the difference between coordinates and dimensions. Otherwise, this tutorial can be very confusing. Dimensions represent degrees of freedom, coordinates are discrete values inside of a dimension.

**Variable**

If a mathematical expression is used to describe the `GenericSeries`, the individual terms of this expression can be divided into two groups. The first group are variables. Variables are symbols that don't get values assigned to them during the creation of a `GenericSeries`. They let us evaluate the expression for differen coordinates. Consider the following expression:

$$
2 \cdot x + 3
$$

Here $x$ is our variable. We can evaluate this expression over and over again by providing differen values for $x$. For example if we would use $x=2$, the result is $7$. With $x=4$ we would get $11$. An important fact to note is that each variable a `GenericSeries`' expression is a dimension. But not every dimension of an expression is necessarily represented by a variable. We will show some code examples later that make this more understandable.

**Parameter**

The second group of therms in an expression based `GenericSeries` are parameters. Parameters are also symbols of an expression, but in contrast to variables, they already get discrete values assigned to them.